## NDVI after the Las Conchas fire in New Mexico

This notebook will be used to further investigate NDVI rasters before and after the Las Conchas fire in New Mexico. We'll also create plot with a slider so that we can see how NDVI recovers in the years after the fire.

### Step 0: Restore variables and import libraries

In [1]:
%store -r ndvi_da fire_bound_gdf

In [2]:
# Import libraries

import earthpy
import hvplot.pandas
import hvplot.xarray


import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import rioxarray as rxr
import xarray as xr

from shapely.geometry import mapping

### Step 1: Assess if NDVI is different inside and outside the fire boundary

In [ ]:
# Check CRSs, make them match if needed.

if ndvi_da.NDVI.rio.crs == fire_bound_gdf.crs:
    print('The CRSs match!')
else:
    print(ndvi_da.NDVI.rio.crs) # EPSG:4326
    print(fire_bound_gdf.crs) # EPSG:4269

# The CRSs match, so we don't need to do anything more here.

The CRSs match!


We need to clip the DataArray into two arrays, so that we can look at NDVI from inside the fire boundary, and outside.

In [7]:
# Clip data to both inside and outside the boundary

# Inner
ndvi_in = ndvi_da.rio.clip(fire_bound_gdf.geometry.apply(mapping))

# Outer
ndvi_out = ndvi_da.rio.clip(
    fire_bound_gdf.geometry.apply(mapping), invert=True
    )

In [9]:
# Quick plots
# We should see the two plots match the clipped area like puzzle pieces.

# Make plots
plot_in = ndvi_in.isel(date=0).hvplot(x='x', y='y', cmap=plt.cm.PiYG, geo=True, 
                   title='NDVI inside Fire Boundary')

plot_out = ndvi_out.isel(date=0).hvplot(x='x', y='y', cmap=plt.cm.PiYG, geo=True, 
                   title='NDVI outside Fire Boundary')

# Show plots
(plot_in + plot_out)

:Layout
   .Image.I  :Image   [x,y]   (NDVI)
   .Image.II :Image   [x,y]   (NDVI)

In [10]:
# Compute mean annual NDVI

# Inside
ndvi_in_mean_df = (ndvi_in
    
    #.sel(date=ndvi_in['date'].dt.month.isin([7])) 
    # Select NDVI data in July; I've commented this out so the 
    # analysis works

    # Now group into years
    .groupby('date.year')

    # Now take the mean across all dimensions
    .mean(...)
    .to_dataframe()
    )

# Outside
ndvi_out_mean_df = (
    ndvi_out#.sel(date=ndvi_out['date'].dt.month.isin([7]))
    .groupby('date.year')
    .mean(...)
    .to_dataframe()
    )

In [12]:
# Join the inside and outside DFs

ndvi_df = (ndvi_in_mean_df[['NDVI']]
           .join(ndvi_out_mean_df[['NDVI']], 
                 lsuffix=' Inside Fire',
                 rsuffix=' Outside Fire'))
ndvi_df

,NDVI Inside Fire,NDVI Outside Fire
year,,
2006,0.536539,0.470742
2007,0.575677,0.502179
2008,0.560684,0.486118
2009,0.569864,0.495966
2010,0.576190,0.491205
2011,0.426936,0.458000
2012,0.377027,0.455526
2013,0.390443,0.447340
2014,0.439131,0.479393


In [15]:
# Plot annual mean NDVI inside and outside on one plot

ins_out_plot_ma_plot = ndvi_df.hvplot(title='Mean Annual NDVI inside and outside\nLas Conchas Fire Boundary')
ins_out_plot_ma_plot

:NdOverlay   [Variable]
   :Curve   [year]   (value)

**PLOT INTERPRETATION HERE**

Now, we'll plot just the difference in NDVI between the two areas on an annual basis.

In [20]:
# Calculate the difference inside and outside
ndvi_df['Difference'] = ndvi_df['NDVI Outside Fire'] - ndvi_df['NDVI Inside Fire']
ndvi_df

,NDVI Inside Fire,NDVI Outside Fire,Difference
year,,,
2006,0.536539,0.470742,-0.065797
2007,0.575677,0.502179,-0.073498
2008,0.560684,0.486118,-0.074566
2009,0.569864,0.495966,-0.073898
2010,0.576190,0.491205,-0.084985
2011,0.426936,0.458000,0.031064
2012,0.377027,0.455526,0.078499
2013,0.390443,0.447340,0.056897
2014,0.439131,0.479393,0.040262


In [26]:
# Plot annual mean NDVI inside and outside on one plot

difference_ma_plot = ndvi_df['Difference'].hvplot(
    title=('Difference in Mean Annual NDVI\n'
    'outside and inside Las Conchas Fire Boundary'),
    ylabel='NDVI')
difference_ma_plot

:Curve   [year]   (Difference)

**PLOT INTERPRETATION HERE**

### Step 2: Create a dynamic plot to see how NDVI changes each year after the fire.